In [1]:
# importing all necessary libraries

import pandas as pd
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
from nilearn import plotting
import os
import cv2 as cv

# Create a function that will create new mask after BEN's generation + filling all missing pixels in the mask

In [2]:
def update_header_and_save(raw_filepath, mask_filepath, new_mask_filepath):
    raw_img = nib.load(raw_filepath)
    mask_img = nib.load(mask_filepath)

    
    mask_data = mask_img.get_fdata()

   
    for i in range(mask_data.shape[2]):

       
        slice_img = mask_data[:, :, i]

        
        slice_img_8bit = cv.normalize(slice_img, None, 0, 255, cv.NORM_MINMAX).astype('uint8')

       
        kernel = np.ones((5, 5), np.uint8)
        closing = cv.morphologyEx(slice_img_8bit, cv.MORPH_CLOSE, kernel)

        
        mask_data[:, :, i] = closing

    attributes = ['extents', 'dim_info', 'slice_end', 'cal_max', 'cal_min', 
                  'glmax', 'glmin', 'srow_x', 'srow_y', 'srow_z']

    header = mask_img.header.copy()
    for attr in attributes:
        header[attr] = raw_img.header[attr]

    new_affine = np.eye(4)
    new_affine[:-1, :] = np.array([header['srow_x'], header['srow_y'], header['srow_z']])

    updated_mask_img = nib.Nifti1Image(mask_data, new_affine, header)
    nib.save(updated_mask_img, new_mask_filepath)

In [3]:
data_dir = '/home/sharapova/test/new_data_cv2'
data_dir

'/home/sharapova/test/new_data_cv2'

In [4]:
raw_dir = os.path.join(data_dir, 'raw')
print(raw_dir)
mask_dir = os.path.join(data_dir, 'mask')
print(mask_dir)
new_mask_dir = os.path.join(data_dir, 'new_mask')
print(new_mask_dir)

/home/sharapova/test/new_data_cv2/raw
/home/sharapova/test/new_data_cv2/mask
/home/sharapova/test/new_data_cv2/new_mask


In [7]:
raw_files = os.listdir(raw_dir)
raw_files

['T2_UT172.nii',
 'T2_UT169.nii',
 'T2_UT175.nii',
 'T2_UT226.nii',
 'T2_UT225.nii',
 'T2_UT171.nii',
 'T2_UT223.nii',
 'T2_UT354.nii',
 'T2_UT356.nii',
 'T2_UT173.nii',
 'T2_UT224.nii',
 'T2_UT230.nii',
 'T2_UT170.nii',
 'T2_UT168.nii',
 'T2_UT376.nii',
 'T2_UT227.nii',
 'T2_UT174.nii',
 'T2_UT229.nii',
 'T2_UT228.nii']

In [8]:
raw_files_mask = os.listdir(mask_dir)
raw_files_mask

['T2_UT172.nii',
 'T2_UT169.nii',
 'T2_UT175.nii',
 'T2_UT226.nii',
 'T2_UT225.nii',
 'T2_UT171.nii',
 'T2_UT223.nii',
 'T2_UT354.nii',
 'T2_UT356.nii',
 'T2_UT173.nii',
 'T2_UT224.nii',
 'T2_UT230.nii',
 'T2_UT170.nii',
 'T2_UT168.nii',
 'T2_UT376.nii',
 'T2_UT227.nii',
 'T2_UT174.nii',
 'T2_UT229.nii',
 'T2_UT228.nii']

In [9]:
new_mask = os.listdir(new_mask_dir)
new_mask

[]

In [10]:
for raw_file in raw_files:
    raw_filepath = os.path.join(raw_dir, raw_file)
    mask_filepath = os.path.join(mask_dir, raw_file)
    new_mask_filepath = os.path.join(new_mask_dir, os.path.splitext(raw_file)[0] + '_mask.nii')
    update_header_and_save(raw_filepath, mask_filepath, new_mask_filepath)

# Testing function to make borders smother for the image mask

In [16]:
img_mask = nib.load('new_data_cv2/mask/T2_UT168.nii')
print(img_mask.header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 128 160  96   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float64
bitpix          : 64
slice_start     : 0
pixdim          : [1.  0.1 0.1 0.1 0.  0.  0.  0. ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 2
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : scanner
sform_code      : scanner
quatern_b       : -0.70710677
quatern_c       : -2.1648903e-17
quatern_d       : 2.1648903e-17
qoffset_x       : -6.75533
qoffset_y       : -5.663706
qoffset_z       : 6.88325
sr